## Real Estate Recommender

### Generating Real Estate Listings

In [38]:
groups = ["young professionals", "families", "retirees and seniors", "investors", "students", "luxury buyers", "first-time homebuyers", "eco-conscious buyers", "high-net-worth individuals", "low-income families", "digital nomads and remote workers", "artists and creatives"]
user_prompt_template = lambda buyer_group: f"""Task: Create a detailed and appealing real estate property listing for {buyer_group}."""
system_prompt_template = """
                        You are an assistant that generate text without repeatation. Text should follow following format strictly:
                        - Neighborhood: address, city
                        - Price: price
                        - Bedrooms: number of bedrooms
                        - Bathrooms number of bathrooms
                        - House Size: size in sqft
                        - Description: general description of the property
                        - Neighborhood Description: description of neighborhood environments
                    """

In [39]:
# Importing the necessary library for OpenAI API
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [53]:

def generate_resl_estate_listing(buyer_group):
    try:
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
            # the model supports same api format as openai does
            base_url="http://localhost:8080"
        )
        response = client.chat.completions.create(
            model="gpt4-turbo",
            # model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt_template,
                },
                {
                    "role": "user",
                    "content": user_prompt_template(buyer_group),
                },
            ],
            temperature=1,
            # max_tokens=256,
            top_p=1,
            frequency_penalty=1,
            presence_penalty=1
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"


In [57]:
import pandas as pd
import time

df_list = []
size_per_group = 10
for id, people in enumerate(groups):
    df_list.append(pd.DataFrame({"listing": [generate_resl_estate_listing(user_prompt_template(people)) for i in range(size_per_group)], "group": [id]*size_per_group}))
    time.sleep(10)
df = pd.concat(df_list).reset_index(drop=True)
pd.set_option("display.max_colwidth", None)
df

,listing,group
0,"- Neighborhood: 250 Waterfront St, Marina District\n- Price: $895,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,100 sqft\n- Description: Immerse yourself in the heart of vibrancy and comfort with this stylish modern condo designed for young professionals. The open-plan living space is bathed in natural light from floor-to-ceiling windows that offer enchanting cityscape views. High-end finishes are evident throughout the kitchen that boasts stainless steel appliances, sleek cabinetry, and a chic breakfast bar perfect for on-the-go mornings or intimate dinners. Both bedrooms serve as private havens with ample closet space and plush carpeting underfoot.\n \n Indulge in the luxury of en-suite bathrooms featuring contemporary fixtures and fine tile work that add to the elegance of urban living. As an added convenience, an in-unit washer and dryer set simplifies laundry days.\n\n This home is more than just its interior; it includes access to top-tier building amenities such as a rooftop terrace with panoramic views, state-of-the-art fitness center, residents' lounge for entertaining guests, secured parking structure, and attentive concierge services enhancing daily life.\n\n- Neighborhood Description: Located in the sought-after Marina Districts area often frequented by thriving professionals. The neighborhood pulses with cosmopolitan energy yet retains a relaxed atmosphere conducive to networking and unwinding alike. Within walking distance are gourmet restaurants offering international cuisines, boutique shops for fashion-forward finds along scenic promenades lined with public art installations reflective of local creativity.\n\n Outdoor enthusiasts will appreciate proximity to parks where jogging trails meander alongside waterfront vistas—a tranquil escape from busy schedules. Cultural institutions like theatres hosting premiere performances contribute to rich experiences just steps away from your front door while reliable public transit connects you effortlessly across the city's diverse neighborhoods.\n",0
1,"- Neighborhood: 352 Parkview Terrace, Austin\n- Price: $425,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,200 sqft\n- Description: This sleek and modern condo offers the perfect blend of style and convenience for the young professional. Featuring state-of-the-art appliances, hardwood flooring throughout, an open-concept living space with large windows that flood the area with natural light. The master suite includes a walk-in closet and a pristine en-suite bathroom. Additional amenities include in-unit laundry facilities and private balcony with breathtaking city views.\n \n- Neighborhood Description: Situated in the heart of Austin's vibrant scene, Parkview Terrace is an ideal spot for those looking to be at arm’s reach from bustling cafes, hip restaurants & bars, boutique shopping options as well as premier entertainment venues. The neighborhood boasts easy access to public transportation facilitating commutes. It is also known for its safety measures and pet-friendly parks which just adds another layer of allure to this dynamic community perfect for social but career-focused individuals.",0
2,"- Neighborhood: 255 Upstart Lane, Silicon Valley\n- Price: $1,200,000\n- Bedrooms: 3\n- Bathrooms: 2\n- House Size: 1800 sqft\n- Description: This modern and sleek single-family home offers an open floor plan with high-end finishes throughout. The kitchen boasts stainless steel appliances and a large island perfect for entertaining. Hardwood floors lead you through the light-filled living space to a cozy backyard oasis. Equipped with smart home features and energy-efficient windows.\n- Neighborhood Description: Situated in the heart of Silicon Valley, this neighborhood is vibrant and full of life, catering to young professionals keen on innovation and convenience. Just steps away from tech headquarters, dynamic start-ups, upscale dining options, boutique coffee shops, and lush parks ideal for weekend

### Storing Listings in a Vector Database

In [61]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
import lancedb
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# func = get_registry().get("openai").create(name="text-embedding-ada-002")
import openai
import os

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
    # the model supports same api format as openai does
    base_url="http://localhost:8080"
)
def embed_func(c):    
    response = client.embeddings.create(input=c, model=EMBEDDING_MODEL_NAME)
    return [data.embedding for data in response.data]

/lhome/yuwewan/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
class RealEstateListing(LanceModel):
    vector: Vector(384)
    listing: str
    group: str

In [63]:
from lancedb.embeddings import with_embeddings

df_embedded = with_embeddings(embed_func, df, column="listing").to_pandas()
df_embedded


/tmp/ipykernel_4049876/369268211.py:3: DeprecationWarning: Function with_embeddings is deprecated and will be removed in a future version
  df_embedded = with_embeddings(embed_func, df, column="listing").to_pandas()
/lhome/yuwewan/anaconda3/envs/py310/lib/python3.10/site-packages/ratelimiter.py:127: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  __aexit__ = asyncio.coroutine(__exit__)


,listing,group,vector
0,"- Neighborhood: 250 Waterfront St, Marina District\n- Price: $895,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,100 sqft\n- Description: Immerse yourself in the heart of vibrancy and comfort with this stylish modern condo designed for young professionals. The open-plan living space is bathed in natural light from floor-to-ceiling windows that offer enchanting cityscape views. High-end finishes are evident throughout the kitchen that boasts stainless steel appliances, sleek cabinetry, and a chic breakfast bar perfect for on-the-go mornings or intimate dinners. Both bedrooms serve as private havens with ample closet space and plush carpeting underfoot.\n \n Indulge in the luxury of en-suite bathrooms featuring contemporary fixtures and fine tile work that add to the elegance of urban living. As an added convenience, an in-unit washer and dryer set simplifies laundry days.\n\n This home is more than just its interior; it includes access to top-tier building amenities such as a rooftop terrace with panoramic views, state-of-the-art fitness center, residents' lounge for entertaining guests, secured parking structure, and attentive concierge services enhancing daily life.\n\n- Neighborhood Description: Located in the sought-after Marina Districts area often frequented by thriving professionals. The neighborhood pulses with cosmopolitan energy yet retains a relaxed atmosphere conducive to networking and unwinding alike. Within walking distance are gourmet restaurants offering international cuisines, boutique shops for fashion-forward finds along scenic promenades lined with public art installations reflective of local creativity.\n\n Outdoor enthusiasts will appreciate proximity to parks where jogging trails meander alongside waterfront vistas—a tranquil escape from busy schedules. Cultural institutions like theatres hosting premiere performances contribute to rich experiences just steps away from your front door while reliable public transit connects you effortlessly across the city's diverse neighborhoods.\n",0,"[0.058508143, 0.02907416, 0.043684054, -0.009807144, -0.050845005, 0.030122882, -0.060871765, 0.0040336815, -0.025515487, -0.0282995, -0.01119739, -0.111601084, 0.03444756, 0.0011670552, 0.08167276, 0.010128735, 0.11821405, 0.026792387, 0.029855762, 0.051701207, -0.04625208, 0.0046300814, 0.00084731093, -0.035731196, 0.0156841, 0.08610467, 0.01831103, 0.03300818, -0.049790993, 0.011102918, 0.059595834, 0.047849704, 0.028044403, -0.06404407, 0.056364294, -0.042862635, -0.034160275, -0.05080546, 0.007422114, -0.00040552652, -0.042594127, -0.017449502, -0.028738149, 0.050229516, -0.045410205, -0.063050054, -0.003511994, 0.03083161, 0.05919047, -0.07009388, 0.03898609, 0.08206192, 0.030433357, 0.03248369, -0.04558283, 0.024247643, -0.040412884, -0.029736688, 0.034573033, -0.044887252, 0.09426998, 0.016264196, 0.020763775, 0.01299787, 0.023354582, -0.03948777, -0.016023932, 0.0987113, 0.012233934, -0.07733977, -0.032462303, -0.0021667096, 0.047857303, 0.04200995, 0.0057053664, 0.009708946, 0.040949054, -0.019909529, -0.00068721455, 0.0044354647, 0.024635343, -0.017793769, -0.03741448, 0.0146732135, 0.011596321, -0.0041640406, 0.00031836514, -0.019088313, 0.036408745, 0.010967304, 0.025408411, -9.308738e-05, -0.15183586, -0.099948965, 0.024020763, -0.0034210996, -0.1385258, -0.02444478, 0.010562316, 0.012679376, ...]"
1,"- Neighborhood: 352 Parkview Terrace, Austin\n- Price: $425,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,200 sqft\n- Description: This sleek and modern condo offers the perfect blend of style and convenience for the young professional. Featuring state-of-the-art appliances, hardwood flooring throughout, an open-concept living space with large windows that flood the area with natural light. The master suite includes a walk-in closet and a pristine en-suite bathroom. Additional amenities include in-unit laundry facilities and private balcony with breathtaking city views.\n \n- 

In [64]:

db = lancedb.connect("./lancedb")
table_name = "real_estate_listings"
table = db.create_table(table_name, schema=RealEstateListing, mode="overwrite")

[2024-06-22T10:25:13Z WARN  lance::dataset] No existing dataset at /lhome/yuwewan/workspace/Minverva/genAI_udacity/Real_Estate_Agent/lancedb/real_estate_listings.lance, it will be created


In [65]:
table.add(df_embedded.to_dict('records'))
table.head().to_pandas()

,vector,listing,group
0,"[0.058508143, 0.02907416, 0.043684054, -0.009807144, -0.050845005, 0.030122882, -0.060871765, 0.0040336815, -0.025515487, -0.0282995, -0.01119739, -0.111601084, 0.03444756, 0.0011670552, 0.08167276, 0.010128735, 0.11821405, 0.026792387, 0.029855762, 0.051701207, -0.04625208, 0.0046300814, 0.00084731093, -0.035731196, 0.0156841, 0.08610467, 0.01831103, 0.03300818, -0.049790993, 0.011102918, 0.059595834, 0.047849704, 0.028044403, -0.06404407, 0.056364294, -0.042862635, -0.034160275, -0.05080546, 0.007422114, -0.00040552652, -0.042594127, -0.017449502, -0.028738149, 0.050229516, -0.045410205, -0.063050054, -0.003511994, 0.03083161, 0.05919047, -0.07009388, 0.03898609, 0.08206192, 0.030433357, 0.03248369, -0.04558283, 0.024247643, -0.040412884, -0.029736688, 0.034573033, -0.044887252, 0.09426998, 0.016264196, 0.020763775, 0.01299787, 0.023354582, -0.03948777, -0.016023932, 0.0987113, 0.012233934, -0.07733977, -0.032462303, -0.0021667096, 0.047857303, 0.04200995, 0.0057053664, 0.009708946, 0.040949054, -0.019909529, -0.00068721455, 0.0044354647, 0.024635343, -0.017793769, -0.03741448, 0.0146732135, 0.011596321, -0.0041640406, 0.00031836514, -0.019088313, 0.036408745, 0.010967304, 0.025408411, -9.308738e-05, -0.15183586, -0.099948965, 0.024020763, -0.0034210996, -0.1385258, -0.02444478, 0.010562316, 0.012679376, ...]","- Neighborhood: 250 Waterfront St, Marina District\n- Price: $895,000\n- Bedrooms: 2\n- Bathrooms: 2\n- House Size: 1,100 sqft\n- Description: Immerse yourself in the heart of vibrancy and comfort with this stylish modern condo designed for young professionals. The open-plan living space is bathed in natural light from floor-to-ceiling windows that offer enchanting cityscape views. High-end finishes are evident throughout the kitchen that boasts stainless steel appliances, sleek cabinetry, and a chic breakfast bar perfect for on-the-go mornings or intimate dinners. Both bedrooms serve as private havens with ample closet space and plush carpeting underfoot.\n \n Indulge in the luxury of en-suite bathrooms featuring contemporary fixtures and fine tile work that add to the elegance of urban living. As an added convenience, an in-unit washer and dryer set simplifies laundry days.\n\n This home is more than just its interior; it includes access to top-tier building amenities such as a rooftop terrace with panoramic views, state-of-the-art fitness center, residents' lounge for entertaining guests, secured parking structure, and attentive concierge services enhancing daily life.\n\n- Neighborhood Description: Located in the sought-after Marina Districts area often frequented by thriving professionals. The neighborhood pulses with cosmopolitan energy yet retains a relaxed atmosphere conducive to networking and unwinding alike. Within walking distance are gourmet restaurants offering international cuisines, boutique shops for fashion-forward finds along scenic promenades lined with public art installations reflective of local creativity.\n\n Outdoor enthusiasts will appreciate proximity to parks where jogging trails meander alongside waterfront vistas—a tranquil escape from busy schedules. Cultural institutions like theatres hosting premiere performances contribute to rich experiences just steps away from your front door while reliable public transit connects you effortlessly across the city's diverse neighborhoods.\n",0
1,"[0.08807147, -0.031950254, 0.11615688, 0.006585816, -0.09464219, 0.030149572, 0.0077909925, -0.021867914, -0.007555809, -0.032976802, 0.012449364, 0.009467583, 0.0921661, 0.007781201, 0.07434154, 0.00878247, 0.09606928, 0.027645389, 0.024280358, 0.052039545, -0.028513487, 0.00895382, -0.0075076977, -0.017512724, -0.058618948, 0.12708355, 0.01685586, -0.012254036, -0.03175031, 0.0036859997, 0.08263985, 4.6791287e-05, 0.04767689, -0.042601336, 0.043040067, 0.011154155, -0.017736562, -0.068274006, -0.00048933615, -0.028498916, -0.064493924, 0.017040381, 0.000983461, 0.0045599947, -0.06

### Building the User Preference Interface

In [66]:
questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
            "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
            "A quiet neighborhood, good local schools, and convenient shopping options.",
            "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
            "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
            "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
        ]

In [67]:

# build an User Portrait embedding could improve the performance
user_prompt_preference = lambda text: f"""Task: extract user preference based on question and answer : {text}."""
system_prompt_preference = """
                        You generate texts about a potential buyer's preference focusing on factors: address, price, number of bedrooms, number of bathrooms, size in sqft, general property information and neighborhood environment.
                    """
def preference_parser(text):
    try:
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
            # the model supports same api format as openai does
            base_url="http://localhost:8080"
        )
        response = client.chat.completions.create(
            model="gpt4-turbo",
            # model='gpt-4'
            messages=[
                {
                    "role": "system",
                    "content": system_prompt_template,
                },
                {
                    "role": "user",
                    "content": user_prompt_template(text),
                },
            ],
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [68]:
QA=""
for i in range(len(questions)):
    QA = questions[i]+" "+answers[i]+" "
preference_text = preference_parser(QA)
preference_text

'- Neighborhood: 352 Maple Avenue, Springfield\n- Price: $475,000\n- Bedrooms: 4\n- Bathrooms: 3\n- House Size: 2,500 sqft\n- Description: This stunning two-story home features a contemporary open floor plan, an updated kitchen with stainless steel appliances and granite countertops, and a cozy fireplace in the living room. Hardwood floors run throughout the main level, leading to a spacious deck overlooking a well-manicured backyard. The master suite boasts a walk-in closet and a luxurious en suite bathroom with a soaking tub and separate shower. An attached two-car garage and full basement offer plenty of storage.\n- Neighborhood Description: The neighborhood strikes a perfect balance for those seeking an urban vibe within a suburban community. Situated with easy access to a reliable bus line, the daily commute is a breeze. The proximity to the major highway simplifies travel and weekend getaways. Cyclists will rejoice at the bike-friendly roads that connect to scenic routes and park

### Searching Based on Preferences

In [69]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os

EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
    # the model supports same api format as openai does
    base_url="http://localhost:8080"
)
def embed_func(c):    
    response = client.embeddings.create(input=c, model=EMBEDDING_MODEL_NAME)
    return [data.embedding for data in response.data]

In [71]:
preference_embedded = embed_func(preference_text)[0]
df_matched = table.search(preference_embedded).metric("cosine").limit(3).to_pandas()
df_matched

,vector,listing,group,_distance
0,"[0.09428801, -0.029741112, 0.05408801, 0.06099471, -0.0644473, 0.0063242847, 0.005091983, 0.033220083, -0.11282814, -0.0022769182, 0.04483426, 0.026657976, 0.025357345, -0.047925863, -0.016434409, 0.012479858, 0.16383027, 0.026881715, 0.034770522, -0.018560521, -0.044493195, -0.06768414, 0.058635857, 0.0030662913, 0.039352097, 0.08310742, 0.035389196, 0.013527663, -0.009239616, 0.013456478, 0.08552442, -0.060007248, 0.008431066, -0.0032055944, 0.1325788, 0.004835384, 0.061669353, 0.02689979, -0.03623925, 0.012792124, 0.048989516, 0.037561655, -0.0054578763, 0.019029966, -0.05293661, -0.016300615, -0.03626389, -0.047210444, 0.08200691, -0.013897293, -0.008169196, 0.10377856, -0.015508607, 0.008904418, -0.049615595, 0.026376868, -0.008705269, 0.019519955, 0.016524531, -0.010840331, -0.015189139, -0.0072681056, -0.05310952, 0.008428012, -0.0137935905, -0.025516018, -0.0933132, 0.004847795, 0.013484318, -0.041654777, -0.021486646, 0.03637316, -0.027202712, -0.054194596, -0.03293786, -0.02287496, 0.06860909, -0.03446116, 0.03196039, 0.017342785, 0.025453705, -0.04788437, -0.060067523, 0.016147377, -0.030211402, -0.02412766, 0.007579449, -0.009179858, 0.09781384, -0.039129157, 0.012532135, 0.043788537, -0.08630334, -0.08531253, -0.04906254, -0.012401386, -0.060821854, -0.118225105, -0.050610837, 0.03794225, ...]","- Neighborhood: 152 Maple Avenue, Springfield\n- Price: $89,000\n- Bedrooms: 2\n- Bathrooms: 1\n- House Size: 850 sqft\n- Description: This cozy and inviting single-story home offers a practical layout designed for comfortable living. It features updated windows that allow for plenty of natural light and energy efficiency. The kitchen is equipped with modern appliances and ample cabinet space. Freshly painted walls and new laminate flooring make this home move-in ready.\n \n- Neighborhood Description:\nThe property sits in a family-friendly neighborhood known for its welcoming community spirit. Local amenities include parks, an elementary school within walking distance, as well as grocery stores tailored to fit a modest budget. Public transportation is readily available with bus routes connecting the area to the broader city network. This peaceful suburb is perfect for families looking for affordability without compromising on quality of life.",9,0.071637
1,"[0.031475075, -0.02533539, 0.0590766, 0.026226189, -0.078657806, 0.01467934, 0.0014894679, 0.03792695, -0.12582387, -0.0019190438, 0.03506697, 0.032421198, 0.0022375018, -0.036206003, -0.014439669, 0.011736196, 0.1477094, 0.053070195, 0.0463011, -0.0025150797, -0.06403725, -0.09068841, 0.09631554, -0.0029187563, 0.04099223, 0.08578478, 0.025001427, 0.014393917, 0.0061547523, 0.015337466, 0.07914607, -0.063076474, 0.024441857, -0.010821007, 0.15013647, 0.014470158, 0.049915597, 0.03039604, -0.04709756, 0.039160494, 0.060892668, 0.08315397, 0.0035691317, 0.014083397, -0.041824833, 0.004655331, -0.05272486, -0.07136524, 0.068926685, 0.03132028, -0.011456639, 0.13445501, -0.003055367, 0.0010369532, -0.0537064, 0.028758327, 0.0021893429, 0.032698188, -0.0014889323, -0.004109491, -0.009266788, -0.009061872, -0.057644438, -0.026573896, -0.03139744, -0.008003757, -0.08626426, 0.052010983, 0.02173763, -0.018020172, -0.009356017, 0.05918129, -0.050884873, -0.056143343, -0.05615338, -0.019151492, 0.045731477, -0.047903437, 0.045486644, 0.017706074, 0.003949629, -0.04183895, -0.04890758, -0.011662438, -0.013436178, -0.022045571, 0.01295635, -0.0010148378, 0.10028542, -0.024631541, 0.004791288, 0.03358303, -0.083548, -0.10133614, -0.058970716, 0.004552882, -0.053455766, -0.12938559, -0.058935862, 0.038049616, ...]","- Neighborhood: 152 Maple Avenue, Springfield\n- Price: $95,000\n- Bedrooms: 3\n- Bathrooms: 1.5\n- House Size: 1100 sqft\n- Description: This charming and affordable single-family home offers a practical floor plan that includes a spacious living room, an eat-in kitchen with modern appliances, generous cabinet space, and durable count

### Personalizing Listing Descriptions

In [134]:

from langchain.chains import ConversationChain

# build an User Portrait embedding could improve the performance
user_prompt_personalization = lambda text, question: f"""
                        Answer the question based on the context below in a compelling, selling and reader-friendly way without altering factual information, 
                        and if the user is not satisfied with current recommendation, please resummarize the description, 
                        and if the question can't be answered based on the context, say "Sorry, I do not have information about that!"
                        
                        Context: {text}.

                        ----
                        Question: {question}
                        Answer:
                    """
system_prompt_personalization = """you are a real estate selling assistance, generate text to match the buyer's preference at best"""
def user_feedback(feedback):
    try:
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
            # the model supports same api format as openai does
            base_url="http://localhost:8080"
        )
        response = client.chat.completions.create(
            model="gpt4-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "you are an user intention assist.",
                },
                {
                    "role": "user",
                    "content": f"""you estimate whether user is satisfied with the current recommendation based on user feedback, please answer only with "positive", "neutral" or "negative". Feedback: {feedback}""",
                },
            ],
            temperature=1,
            # max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

def personalize_bot(listing, question):
    try:
        client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            # I have chosen to use local model host, as OpenAI has recently closed the 5$ api key bonus
            # the model supports same api format as openai does
            base_url="http://localhost:8080"
        )
        response = client.chat.completions.create(
            model="gpt4-turbo",
            messages=[
                {
                    "role": "system",
                    "content": system_prompt_personalization,
                },
                {
                    "role": "user",
                    "content": user_prompt_personalization(listing, question),
                },
            ],
            temperature=1,
            # max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

index = 0
def run_bot(question):
    global index
    if user_feedback(question) == 'negative': 
        if index < (df_matched.shape[0]-1):
            index += 1
            return personalize_bot(df_matched.loc[index, "listing"], "do you have any recommandation?")
        else:
            return "Sorry, I could not find a match for your!"
    else: 
        return personalize_bot(df_matched.loc[index, "listing"], question)

In [126]:
question = "do you have any recommandation?"
answer = run_bot(question)
answer

"Absolutely, I would be delighted to recommend the charming home located at 152 Maple Avenue in Springfield. Priced at an accessible $89,000, this warm and inviting residence is an excellent match for those seeking a cozy, yet efficient living space. With two well-sized bedrooms and one full bathroom, it suits small families, couples, or individuals looking for a comfortable abode.\n\nThe house is set on a modest 850 sqft footprint, making it easy to maintain without sacrificing the convenience of a thoughtfully arranged floor plan. The updated windows not only flood the home with natural sunlight, creating a bright and airy atmosphere but also contribute to energy savings year-round. In the heart of the home, the kitchen is well-equipped with modern appliances and boasts ample cabinet space, perfect for those who enjoy cooking and baking.\n\nNew laminate flooring and freshly painted walls ensure the new owner will be able to move in without worrying about immediate renovations. It's a

In [127]:
question_1 = "how much does it cost?"
answer_1 = run_bot(question_1)
answer_1

'The inviting home you are inquiring about at 152 Maple Avenue, Springfield, is priced at an affordable $89,000. This presents an exceptional value for a well-maintained, move-in ready property in a family-friendly neighborhood.'

In [128]:
question_2 = "is a garage inclusive?"
answer_2 = personalize_bot(df_matched.loc[0,"listing"], question_2)
answer_2

'Sorry, I do not have information about that!'

In [135]:
run_bot("could you recommend me another object?")

"Absolutely, I'd love to recommend a home that may just be perfect for you!\n\nNestled in the heart of the friendly, community-oriented neighborhood of 152 Maple Avenue, Springfield, is a charming and affordably priced single-family home waiting for you to move in. Priced at just $95,000, this delightful property boasts three cozy bedrooms and one and a half bathrooms, making it ideal for a small family or anyone looking to downsize without compromising on living space.\n\nThe house itself encompasses 1100 sqft of well-designed living space that includes a spacious living room where you can relax or entertain guests. The eat-in kitchen is a cook's delight, featuring modern appliances, generous cabinet storage, and durable countertops that are sure to withstand the hustle and bustle of daily life. The bedrooms are flooded with natural light, creating a serene ambiance, and each comes with ample closet space to keep your belongings neatly organized. \n\nTo add to the appeal, the home is 